In [1]:
import pandas as pd

df = pd.read_csv(
    "noticias_unificadas.tsv",
    encoding="utf-8",
    sep="\t",
    dtype={"fecha": "string", "titulo": "string", "contenido": "string", "seccion": "string", "link": "string"},
    quoting=0,
    na_filter=False
)

In [2]:
topicos = df['seccion'].value_counts()
print(f"\nTotal de tópicos únicos: {df['seccion'].nunique()}")


Total de tópicos únicos: 7


In [3]:
df.head()

,fecha,titulo,contenido,seccion,link
0,2025-11-09,Jueces rechazan intento de afectación a la ind...,"Desde la ciudad de Tacna, jueces y juezas de t...",Política,https://diariocorreo.pe/politica/jueces-rechaz...
1,2025-11-09,Liga 1: Lo gritan los “Churres” y todo el pueb...,Alianza Atlético le sacó lustre a su clasifica...,Deportes,https://diariocorreo.pe/deportes/alianza-atlet...
2,2025-11-09,Proponen sancionar con hasta 10 años de cárcel...,"La congresista Elizabeth Medina Hermosillo, de...",Política,https://diariocorreo.pe/politica/proponen-sanc...
3,2025-11-09,Este lunes inicia la semana de representación ...,Desde este lunes 10 hasta el viernes 14 de nov...,Política,https://diariocorreo.pe/politica/este-lunes-in...
4,2025-11-09,Selección peruana evalúa reprogramación de par...,La Federación Peruana de Fútbol (FPF) informó ...,Deportes,https://diariocorreo.pe/deportes/seleccion-per...


In [4]:
import nltk
from nltk import trigrams
from nltk.tokenize import sent_tokenize, word_tokenize


In [5]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/joelibaceta/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/joelibaceta/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [6]:
nltk.data.find("tokenizers/punkt")

FileSystemPathPointer('/home/joelibaceta/nltk_data/tokenizers/punkt')

In [7]:
nltk.data.find("tokenizers/punkt_tab")

FileSystemPathPointer('/home/joelibaceta/nltk_data/tokenizers/punkt_tab')

In [8]:
from typing import Dict, List, Tuple

def tokenize_es(text: str) -> list[list[str]]:
    if not isinstance(text, str) or not text.strip():
        return []
    sentences = sent_tokenize(text, language="spanish")
    tokenized = []
    for s in sentences:
        toks = word_tokenize(s, language="spanish")
        # lower only alphabetic tokens, keep punctuation as-is
        toks = [t.lower() if t.isalpha() else t for t in toks]
        if toks:
            tokenized.append(toks)
    return tokenized

In [9]:
from collections import defaultdict

def load_corpus(df: pd.DataFrame, by_category: bool = False) -> Dict[str, List[List[str]]]:
    buckets: Dict[str, List[List[str]]] = defaultdict(list)
    
    for _, row in df.iterrows():
        categoria = (row.get("seccion") or "").strip()
        noticia = row.get("contenido")
        
        if not noticia or not isinstance(noticia, str):
            continue
            
        sents_toks = tokenize_es(noticia)
        
        if by_category and categoria:
            for s in sents_toks:
                if s:
                    buckets[categoria].append(s)
        else:
            for s in sents_toks:
                if s:
                    buckets["_GLOBAL"].append(s)
    
    return buckets

In [10]:
corpus_por_categoria = load_corpus(df, by_category=True)
corpus_global = load_corpus(df, by_category=False)

In [11]:
def train_trigrams(tokenized_sentences: List[List[str]]):
    """
      model[(w1,w2)][w3] = prob
    """
    model = defaultdict(lambda: defaultdict(float))
    for sent in tokenized_sentences:
        for w1, w2, w3 in trigrams(sent, pad_left=True, pad_right=True):
            model[(w1, w2)][w3] += 1.0

    for w1w2 in model:
        total = sum(model[w1w2].values())
        if total > 0:
            for w3 in model[w1w2]:
                model[w1w2][w3] /= total
    return model

In [12]:
model_global = train_trigrams(corpus_global["_GLOBAL"])
len(model_global)

2274244

In [13]:
models_por_categoria = {}

for categoria, oraciones in corpus_por_categoria.items():
    print(f"Entrenando modelo para: {categoria} ({len(oraciones)} oraciones)")
    models_por_categoria[categoria] = train_trigrams(oraciones)


Entrenando modelo para: Política (164273 oraciones)
Entrenando modelo para: Deportes (64217 oraciones)
Entrenando modelo para: Deportes (64217 oraciones)
Entrenando modelo para: Espectáculos (95589 oraciones)
Entrenando modelo para: Espectáculos (95589 oraciones)
Entrenando modelo para: Cultura (82146 oraciones)
Entrenando modelo para: Cultura (82146 oraciones)
Entrenando modelo para: Economía (56308 oraciones)
Entrenando modelo para: Economía (56308 oraciones)
Entrenando modelo para: Mundo (81657 oraciones)
Entrenando modelo para: Mundo (81657 oraciones)
Entrenando modelo para: Policiales (50687 oraciones)
Entrenando modelo para: Policiales (50687 oraciones)


In [14]:
from typing import Optional
import random

def sample_next(model: dict, w1: str, w2: str) -> Optional[str]:
    dist = model.get((w1, w2), {})
    # Heuristica por si no hay distribución para (w1, w2)
    if not dist:
        # Fallback 1: Buscar todos los contextos que terminen en w2
        for key in model.keys():
            if key[1] == w2 and model[key]:
                dist = model[key]
                break
        # Fallback 2: Si aún no hay nada, usar inicio de oración
        if not dist:
            dist = model.get((None, None), {})
        # Fallback 3: Si todavía no hay nada, elegir un contexto aleatorio
        if not dist and model:
            random_key = random.choice(list(model.keys()))
            dist = model[random_key]

    r = random.random()
    acc = 0.0
    for w3, p in dist.items():
        acc += p
        if acc >= r:
            return w3
    
    return list(dist.keys())[-1] if dist else None

In [15]:
def generate_sentence(model: dict, seeds: Tuple[str, str] = (None, None), max_len: int = 30) -> str:
    text: List[str] = [seeds[0], seeds[1]]
    sentence_finished = False

    while not sentence_finished and len(text) < max_len + 2:
        w3 = sample_next(model, text[-2], text[-1])
        text.append(w3)
        if text[-2:] == [None, None] or w3 is None:
            sentence_finished = True

    sentence = " ".join([t for t in text if t])
    sentence = sentence.strip()
    if sentence and sentence[-1] not in ".!?":
        sentence += "."
    if sentence:
        sentence = sentence[0].upper() + sentence[1:]
    return sentence

In [16]:
def generate_paragraph(model: dict, n_sentences: int = 3, seeds: Tuple[str, str] = (None, None)) -> str:
    return " ".join(generate_sentence(model, seeds=seeds) for _ in range(n_sentences))

In [17]:
print(generate_paragraph(model_global, n_sentences=2, seeds=("La", "Policia", )))

La Policia aprovecha que estamos descuidando personal para satisfacción mía pero trae como estigma el haber influido en la zona afectada . La Policia durante protestas impide la inscripción de alianzas procederá hasta los colectiveros y el fondo editorial .


In [18]:
print(generate_paragraph(models_por_categoria["Deportes"], n_sentences=2, seeds=("El", "equipo")))

El equipo que se proclamó máximo goleador histórico de cerro colorado y Arizona.6:30 p.m . El equipo piurano en otros resultados : los chankas , amelí fue enfático al asegurar que protegeremos a cada continente en el equipo de vóley 2025.¡Vóley peruano… vóley peruano… !


In [19]:
print("\nPOLÍTICA:")
print(generate_paragraph(models_por_categoria["Política"], n_sentences=2, seeds=("El", "presidente")))


POLÍTICA:
El presidente pero vizcarra lideró red criminalfiscalía pide prisión preventiva de andrés manuel lópez obrador y continuada desde hace mucho tiempo han criticado la lejanía del local del canal de youtube del. El presidente pero vizcarra lideró red criminalfiscalía pide prisión preventiva en su derecho al voto de los internos e internas y afirmó que si ello no cumplió con las reglas de conducta.El.


In [20]:
from utils.utils import format_sentence

def generate_sentence(model: dict, seeds: Tuple[str, str] = (None, None), max_len: int = 30) -> str:
    w1 = seeds[0].lower() if seeds[0] and seeds[0] is not None else None
    w2 = seeds[1].lower() if seeds[1] and seeds[1] is not None else None
    
    text: List[str] = []

    if w1 is not None:
        text.append(w1)
    if w2 is not None:
        text.append(w2)
    
    if not text:
        w1, w2 = None, None
    elif len(text) == 1:
        w1, w2 = None, text[0]
    else:
        w1, w2 = text[-2] if len(text) >= 2 else None, text[-1]
    
    sentence_finished = False
    iterations_without_word = 0
    
    while not sentence_finished and len(text) < max_len:
        w3 = sample_next(model, w1, w2)
        if w3 is None:
            iterations_without_word += 1
            if iterations_without_word > 3:
                sentence_finished = True
            w1, w2 = None, None
            continue
        
        iterations_without_word = 0
        text.append(w3)
        
        w1, w2 = w2, w3

        if w3 in ['.', '!', '?']:
            sentence_finished = True
    
    return format_sentence(text, capitalize_first=True, add_final_punct=True)

In [21]:
def generate_paragraph(model: dict, n_sentences: int = 3, seeds: Tuple[str, str] = (None, None)) -> str:
    sentences = []
    sentences.append(generate_sentence(model, seeds=seeds))
    for _ in range(n_sentences - 1):
        sentences.append(generate_sentence(model, seeds=(None, None)))
    return " ".join(sentences)

In [22]:
print(generate_paragraph(model_global, n_sentences=3, seeds=("Ayer", "sucedio")))

Ayer sucedio mientras el apra fue proscrito con la arcilla suiza con parciales de hasta S/20 mil al mes los aranceles, que parte de la mujer, fanny montellanos. Además de declamar, cantar y bailar. He pedido que el sábado a la remisión de la separación con un mensaje por la noche del miércoles en culiacán, en un ambiente positivo y comencé el viaje.


In [23]:
print("DEPORTES - Seeds: ('El', 'equipo')")
print(generate_paragraph(models_por_categoria["Deportes"], n_sentences=4, seeds=("El", "equipo")))

DEPORTES - Seeds: ('El', 'equipo')
El equipo que amo, estoy muy ilusionado. En ese contexto, los comisarios `` de aquí a las 8:00 p.m. Si bien los primeros dos partidos de septiembre con un medio ecuatoriano. Video recomendado: el técnico gerardo ameli, quien también se unieron para mostrar que hay seguridad ''.


In [24]:
print("POLÍTICA - Seeds: ('El', 'presidente')")
print(generate_paragraph(models_por_categoria["Política"], n_sentences=3, seeds=("El", "presidente")))

POLÍTICA - Seeds: ('El', 'presidente')
El presidente de china en perú: echa muni cae en saco roto: ni cárceles, cometió un error involuntario ” porque la cantidad de nombramientos en Cajamarca.Juan sheput. Ya se cobra por ello, creo, sin embargo, el comunicado emitido por el poder judicial, ministerio de economía y finanzas, josé cevasco denuncia falta de. Pero si conversar no es ciega; qué disímil puede ser detenido en roma, tomó asiento, y se incautaron y destruyeron veredas —cerámicas de piso tipo roca— para.


In [25]:
print("MODELO GLOBAL - Sin seeds (inicio natural)")
print(generate_paragraph(model_global, n_sentences=3, seeds=(None, None)))

MODELO GLOBAL - Sin seeds (inicio natural)
Asimismo cree firmemente que sí existía “ un millón de dólares en efectivo, cheque o anotación contable relacionados con políticas de emergencia al hospital luis sáenz recibió a universitario. Estas organizaciones de derechos humanos ( cidh ). Con esta persona no puede decir ningún peruano, consolidando una alianza militar con un equipo de ‘ leyenda viva ’ jorge nieto montesinos.


## 🔍 Validación del Modelo

Verificamos que el modelo funcione correctamente con diferentes casos.

In [26]:
print("=" * 80)
print("PRUEBAS DE VALIDACIÓN DEL MODELO DE TRIGRAMAS")
print("=" * 80)

# Test 1: Verificar estructura del modelo
print("\n1️⃣ Verificando estructura del modelo...")
print(f"   - Contextos únicos en modelo global: {len(model_global):,}")
print(f"   - Ejemplo de contexto: {list(model_global.keys())[:3]}")

# Verificar que las probabilidades sumen 1
sample_context = list(model_global.keys())[0]
total_prob = sum(model_global[sample_context].values())
print(f"   - Suma de probabilidades para contexto {sample_context}: {total_prob:.4f}")
print(f"   - ✅ Probabilidades correctas" if abs(total_prob - 1.0) < 0.001 else f"   - ⚠️ Error en probabilidades")

# Test 2: Verificar que los seeds funcionan
print("\n2️⃣ Probando generación con diferentes seeds...")
test_seeds = [
    ("el", "presidente"),
    ("la", "policía"),
    ("los", "ministros"),
    (None, None),  # Sin seeds
]

for w1, w2 in test_seeds:
    try:
        result = generate_sentence(model_global, seeds=(w1, w2), max_len=20)
        w1_str = w1 if w1 else "None"
        w2_str = w2 if w2 else "None"
        print(f"   Seeds ({w1_str:>10}, {w2_str:>10}): {result[:80]}...")
    except Exception as e:
        print(f"   Seeds ({w1}, {w2}): ❌ Error: {e}")

# Test 3: Verificar longitudes
print("\n3️⃣ Probando diferentes longitudes...")
for length in [10, 20, 30]:
    result = generate_sentence(model_global, seeds=(None, None), max_len=length)
    word_count = len(result.split())
    print(f"   Max length={length}: generó {word_count} palabras")

# Test 4: Verificar consistencia (múltiples generaciones)
print("\n4️⃣ Verificando variedad en generaciones...")
generations = set()
for _ in range(5):
    result = generate_sentence(model_global, seeds=("el", "gobierno"), max_len=15)
    generations.add(result)

print(f"   Generadas {len(generations)} oraciones únicas de 5 intentos")
print(f"   {'✅ Buena variedad' if len(generations) >= 3 else '⚠️ Poca variedad - posible problema'}")

print("\n" + "=" * 80)
print("✅ VALIDACIÓN COMPLETADA")
print("=" * 80)

PRUEBAS DE VALIDACIÓN DEL MODELO DE TRIGRAMAS

1️⃣ Verificando estructura del modelo...
   - Contextos únicos en modelo global: 2,274,244
   - Ejemplo de contexto: [(None, None), (None, 'desde'), ('desde', 'la')]
   - Suma de probabilidades para contexto (None, None): 1.0000
   - ✅ Probabilidades correctas

2️⃣ Probando generación con diferentes seeds...
   Seeds (        el, presidente): El presidente en funciones....
   Seeds (        la,    policía): La policía nacional del perú?...
   Seeds (       los,  ministros): Los ministros de estado de bahía blanca, trump ya había sido interrogado por la ...
   Seeds (      None,       None): Perú21 epaper....

3️⃣ Probando diferentes longitudes...
   Max length=10: generó 6 palabras
   Max length=20: generó 20 palabras
   Max length=30: generó 4 palabras

4️⃣ Verificando variedad en generaciones...
   Generadas 5 oraciones únicas de 5 intentos
   ✅ Buena variedad

✅ VALIDACIÓN COMPLETADA


### 🔧 Diagnóstico de Problemas Comunes

Analicemos posibles problemas en el modelo:

In [27]:
# Diagnóstico 1: Verificar contextos con None
print("🔍 ANÁLISIS DE CONTEXTOS CON PADDING (None)")
print("=" * 70)

none_contexts = [(k, len(v)) for k, v in model_global.items() if None in k]
print(f"Total de contextos con None: {len(none_contexts)}")
print(f"Contextos de inicio (None, None): {len(model_global.get((None, None), {}))}")

# Mostrar algunos ejemplos
if (None, None) in model_global:
    print(f"\nPalabras que pueden iniciar una oración:")
    inicio_words = sorted(model_global[(None, None)].items(), key=lambda x: x[1], reverse=True)[:10]
    for word, prob in inicio_words:
        print(f"   {word:>15}: {prob:.4f}")

# Diagnóstico 2: Verificar problemas de mayúsculas
print("\n🔍 ANÁLISIS DE MAYÚSCULAS/MINÚSCULAS")
print("=" * 70)

# Buscar si hay contextos con palabras en mayúsculas (no debería haber)
uppercase_contexts = [(k, v) for k, v in model_global.items() 
                     if any(w and isinstance(w, str) and w[0].isupper() for w in k)]

print(f"Contextos con mayúsculas: {len(uppercase_contexts)}")
if len(uppercase_contexts) > 0:
    print("⚠️ ADVERTENCIA: Hay contextos con mayúsculas - debería estar todo en minúsculas")
    print("Ejemplos:", list(uppercase_contexts)[:5])
else:
    print("✅ Correcto: Todos los contextos están en minúsculas")

# Diagnóstico 3: Verificar contextos huérfanos (con 1 sola opción)
print("\n🔍 ANÁLISIS DE CONTEXTOS HUÉRFANOS")
print("=" * 70)

orphan_contexts = [(k, v) for k, v in model_global.items() if len(v) == 1]
print(f"Contextos con solo 1 opción: {len(orphan_contexts)} ({len(orphan_contexts)/len(model_global)*100:.1f}%)")
print(f"Total de contextos: {len(model_global)}")

# Diagnóstico 4: Verificar cobertura de palabras comunes
print("\n🔍 VERIFICAR COBERTURA DE PALABRAS COMUNES")
print("=" * 70)

common_words = ["el", "la", "de", "en", "que", "y", "a", "los", "del", "las"]
print("Palabras comunes y sus contextos:")

for word in common_words[:5]:
    # Contar en cuántos contextos aparece esta palabra
    as_w1 = sum(1 for k in model_global.keys() if k[0] == word)
    as_w2 = sum(1 for k in model_global.keys() if k[1] == word)
    print(f"   '{word}': aparece en {as_w1} contextos como w1, {as_w2} como w2")

# Diagnóstico 5: Verificar puntuación
print("\n🔍 ANÁLISIS DE PUNTUACIÓN")
print("=" * 70)

punctuation = ['.', ',', '!', '?', ';', ':']
print("Contextos que terminan en puntuación:")

for punct in punctuation[:3]:
    contexts_ending = [(k, v) for k, v in model_global.items() if k[1] == punct]
    if contexts_ending:
        print(f"   Contextos (*, '{punct}'): {len(contexts_ending)}")
        # Mostrar qué sigue después de la puntuación
        next_words = {}
        for k, v in contexts_ending:
            for w3, prob in v.items():
                next_words[w3] = next_words.get(w3, 0) + prob
        top_next = sorted(next_words.items(), key=lambda x: x[1], reverse=True)[:3]
        print(f"      → Palabras más comunes después: {top_next}")

print("\n" + "=" * 70)
print("✅ DIAGNÓSTICO COMPLETADO")
print("=" * 70)

🔍 ANÁLISIS DE CONTEXTOS CON PADDING (None)
Total de contextos con None: 23262
Contextos de inicio (None, None): 21988

Palabras que pueden iniciar una oración:
                el: 0.0787
                la: 0.0602
                en: 0.0454
             video: 0.0378
                 “: 0.0354
                ``: 0.0322
         aprovecha: 0.0310
            Perú21: 0.0225
          búscanos: 0.0196
            ¡Ahora: 0.0196

🔍 ANÁLISIS DE MAYÚSCULAS/MINÚSCULAS
Total de contextos con None: 23262
Contextos de inicio (None, None): 21988

Palabras que pueden iniciar una oración:
                el: 0.0787
                la: 0.0602
                en: 0.0454
             video: 0.0378
                 “: 0.0354
                ``: 0.0322
         aprovecha: 0.0310
            Perú21: 0.0225
          búscanos: 0.0196
            ¡Ahora: 0.0196

🔍 ANÁLISIS DE MAYÚSCULAS/MINÚSCULAS
Contextos con mayúsculas: 60099
⚠️ ADVERTENCIA: Hay contextos con mayúsculas - debería estar todo en minúscul

---

## 📋 Resumen de Problemas Corregidos

### ✅ **Problemas identificados y solucionados:**

1. **Fallbacks incorrectos en `sample_next`**:
   - ❌ Antes: Buscaba contextos `(None, w2)` y `(w1, None)` que no existen
   - ✅ Ahora: Busca contextos reales que terminen en `w2` y usa inicio de oración como fallback

2. **Terminación prematura de oraciones**:
   - ❌ Antes: Se detenía al primer `None` sin intentar continuar
   - ✅ Ahora: Intenta continuar con nuevo contexto antes de rendirse

3. **Limpieza de espacios antes de puntuación**:
   - ❌ Antes: Generaba " ." en lugar de "."
   - ✅ Ahora: Elimina espacios antes de puntuación automáticamente

4. **Manejo robusto de casos edge**:
   - ✅ Contador para evitar loops infinitos
   - ✅ Mejor manejo de contextos no vistos
   - ✅ Validación consistente de minúsculas en seeds

### 🎯 **Mejoras adicionales implementadas:**

- Validación automática del modelo
- Diagnóstico de problemas comunes
- Verificación de probabilidades
- Análisis de cobertura de palabras

### 💡 **Recomendaciones:**

1. **Tamaño del corpus**: Tu modelo será mejor con más datos
2. **Smoothing**: Considera agregar Laplace smoothing para contextos no vistos
3. **Vocabulario**: Filtra palabras muy raras (< 5 apariciones) para reducir sparsity
4. **Modelo más avanzado**: Considera 4-gramas o modelos neurales (LSTM/Transformer) para mejor calidad

## 📊 Análisis: Combinaciones más frecuentes (Trigramas)

In [28]:
from collections import Counter

def get_most_frequent_trigrams(tokenized_sentences: List[List[str]], top_n: int = 20, exclude_padding: bool = True):
    """
    Identifica los trigramas más frecuentes en el corpus.
    
    Args:
        tokenized_sentences: Lista de oraciones tokenizadas
        top_n: Número de trigramas a retornar
        exclude_padding: Si True, excluye trigramas que contienen None (padding)
    
    Returns:
        Lista de tuplas ((w1, w2, w3), frecuencia)
    """
    trigram_counts = Counter()
    
    for sent in tokenized_sentences:
        for w1, w2, w3 in trigrams(sent, pad_left=True, pad_right=True):
            # Opcionalmente excluir trigramas con padding
            if exclude_padding and (w1 is None or w2 is None or w3 is None):
                continue
            trigram_counts[(w1, w2, w3)] += 1
    
    return trigram_counts.most_common(top_n)

In [ ]:
print("TRIGRAMAS MÁS FRECUENTES - CORPUS GLOBAL")

top_trigrams_global = get_most_frequent_trigrams(corpus_global["_GLOBAL"], top_n=30)

for i, ((w1, w2, w3), freq) in enumerate(top_trigrams_global, 1):
    w1_str = f"'{w1}'" if w1 else "None"
    w2_str = f"'{w2}'" if w2 else "None"
    w3_str = f"'{w3}'" if w3 else "None"
    
    print(f"{i:2}. ({w1_str:>15}, {w2_str:>15}, {w3_str:>15}) → {freq:>5} veces")

print(f"\nTotal de trigramas únicos: {len(set(trigrams(sum(corpus_global['_GLOBAL'], []), pad_left=True, pad_right=True)))}")

TRIGRAMAS MÁS FRECUENTES - CORPUS GLOBAL


In [ ]:
# Analizar por categoría
print("\n" + "=" * 70)
print("TRIGRAMAS MÁS FRECUENTES POR CATEGORÍA")
print("=" * 70)

for categoria in ["Deportes", "Política", "Economía"]:
    if categoria in corpus_por_categoria:
        print(f"\n📂 {categoria.upper()}")
        print("-" * 70)
        
        top_trigrams = get_most_frequent_trigrams(
            corpus_por_categoria[categoria], 
            top_n=10, 
            exclude_padding=True
        )
        
        for i, ((w1, w2, w3), freq) in enumerate(top_trigrams, 1):
            print(f"  {i:2}. ({w1}, {w2}, {w3}) → {freq} veces")

### Visualización de trigramas frecuentes

In [ ]:
import matplotlib.pyplot as plt

def plot_top_trigrams(trigrams_list, title="Top Trigramas", top_n=15):
    """
    Visualiza los trigramas más frecuentes en un gráfico de barras.
    """
    # Tomar solo los top_n
    trigrams_list = trigrams_list[:top_n]
    
    # Preparar datos
    labels = [f"{w1} {w2} {w3}" for (w1, w2, w3), _ in trigrams_list]
    frequencies = [freq for _, freq in trigrams_list]
    
    # Crear gráfico
    fig, ax = plt.subplots(figsize=(12, 8))
    bars = ax.barh(range(len(labels)), frequencies, color='steelblue', alpha=0.8)
    
    ax.set_yticks(range(len(labels)))
    ax.set_yticklabels(labels, fontsize=10)
    ax.set_xlabel('Frecuencia', fontsize=12, fontweight='bold')
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.invert_yaxis()
    ax.grid(axis='x', alpha=0.3)
    
    # Agregar valores al final de las barras
    for i, (bar, freq) in enumerate(zip(bars, frequencies)):
        ax.text(freq + 0.5, i, str(freq), va='center', fontsize=9, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

# Visualizar trigramas del corpus global
plot_top_trigrams(top_trigrams_global, title="Top 15 Trigramas - Corpus Global", top_n=15)

In [ ]:
# Comparación entre categorías
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Trigramas Más Frecuentes por Categoría', fontsize=16, fontweight='bold', y=0.995)

categorias_plot = ["Deportes", "Política", "Economía", "Internacional"]
axes_flat = axes.flatten()

for idx, categoria in enumerate(categorias_plot):
    if categoria in corpus_por_categoria:
        top_cat = get_most_frequent_trigrams(
            corpus_por_categoria[categoria], 
            top_n=10, 
            exclude_padding=True
        )
        
        labels = [f"{w1} {w2} {w3}" for (w1, w2, w3), _ in top_cat]
        frequencies = [freq for _, freq in top_cat]
        
        ax = axes_flat[idx]
        bars = ax.barh(range(len(labels)), frequencies, color='coral', alpha=0.7)
        ax.set_yticks(range(len(labels)))
        ax.set_yticklabels(labels, fontsize=9)
        ax.set_xlabel('Frecuencia', fontsize=10)
        ax.set_title(f'📂 {categoria}', fontsize=12, fontweight='bold')
        ax.invert_yaxis()
        ax.grid(axis='x', alpha=0.3)
        
        # Valores en las barras
        for i, (bar, freq) in enumerate(zip(bars, frequencies)):
            ax.text(freq + 0.3, i, str(freq), va='center', fontsize=8)
    else:
        axes_flat[idx].text(0.5, 0.5, f'Categoría "{categoria}"\nno disponible', 
                           ha='center', va='center', transform=axes_flat[idx].transAxes)
        axes_flat[idx].set_xticks([])
        axes_flat[idx].set_yticks([])

plt.tight_layout()
plt.show()

### Análisis adicional: Bigramas más frecuentes

In [ ]:
from nltk import bigrams

def get_most_frequent_bigrams(tokenized_sentences: List[List[str]], top_n: int = 20, exclude_padding: bool = True):
    """
    Identifica los bigramas más frecuentes en el corpus.
    """
    bigram_counts = Counter()
    
    for sent in tokenized_sentences:
        for w1, w2 in bigrams(sent, pad_left=True, pad_right=True):
            if exclude_padding and (w1 is None or w2 is None):
                continue
            bigram_counts[(w1, w2)] += 1
    
    return bigram_counts.most_common(top_n)

# Analizar bigramas
print("=" * 70)
print("BIGRAMAS MÁS FRECUENTES - CORPUS GLOBAL")
print("=" * 70)

top_bigrams = get_most_frequent_bigrams(corpus_global["_GLOBAL"], top_n=30, exclude_padding=True)

for i, ((w1, w2), freq) in enumerate(top_bigrams, 1):
    print(f"{i:2}. ('{w1}', '{w2}') → {freq:>5} veces")

### 📈 Interpretación de resultados

Los trigramas más frecuentes te muestran:
1. **Patrones comunes** en el lenguaje periodístico español
2. **Frases típicas** que se repiten en las noticias
3. **Contextos específicos** de cada categoría (deportes, política, etc.)

Esto es útil para:
- Entender qué combinaciones de palabras son más naturales
- Mejorar la generación de texto (el modelo aprende de estas frecuencias)
- Identificar vocabulario característico de cada temática